----------------------------------------------------------------------------------------------------------------------

In [ ]:
# Eğer yüklü değilse terminalden şunu çalıştır:
# %pip install -U langchain-community chromadb python-dotenv
%pip install langgraph

from dotenv import load_dotenv
import os
import requests
import time
import pyttsx3

from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode


load_dotenv()  # .env dosyasından API anahtarını yükler


----------------------------------------------------------------------------------------------------------------------

In [ ]:
LLM = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1,
)

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)

qa_chain = RetrievalQA.from_chain_type(
    llm=LLM,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff"
)


----------------------------------------------------------------------------------------------------------------------

In [ ]:
ENDPOINT_ATTENTION = "http://127.0.0.1:8001/attention"
ENDPOINT_SCRIPT = "http://localhost:5000/get_texts"
ENDPOINT_KYBRD_MOUSE_INTERRUPT = "http://localhost:5001/api/status"

----------------------------------------------------------------------------------------------------------------------

In [ ]:
from datetime import datetime

def dikkat_uyarisi_ver(_):
    try:
        recent = vectordb.similarity_search("attention", k=1)[0]
        data = recent.metadata
        if not data["screen"] or data["attention"] < 0.5:
            print("⚠️ Dikkatin dağıldı! Lütfen odaklan.")
            try:
                import winsound
                winsound.Beep(1000, 400)
            except:
                pass
            print("\033[91m[Çerçeve RENGİ: KIRMIZI - ODAK YOK]\033[0m")
    except Exception as e:
        print(f"🚨 Dikkat modülü hatası: {e}")


def mola_onerisi_chromadb(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        att_20s = [doc.metadata.get("att_20min") for doc in recent_docs if isinstance(doc.metadata.get("att_20min"), (float, int))]
        if len(att_20s) >= 3 and sum(att_20s)/len(att_20s) < 0.6:
            print("😴 20dk ortalaman düşük. Mola vermelisin.")
            try:
                import winsound
                winsound.Beep(800, 500)
            except:
                pass
            print("\033[95m[Mola Önerisi: Düşük dikkat algılandı.]\033[0m")
    except Exception as e:
        print(f"🚨 Mola modülü hatası: {e}")


def periyot_onerisi_llm_ileri(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=15)
        att_vals = [doc.metadata.get("attention", 0) for doc in recent_docs]
        focus_vals = [doc.metadata.get("focus_score", 0) for doc in recent_docs]
        attention_summary = "\n".join([
            f"- Dikkat: {round(doc.metadata.get('attention', 0),2)} | 20dk: {round(doc.metadata.get('att_20min', 0),2)} | Focus: {round(doc.metadata.get('focus_score', 0),2)}"
            for doc in recent_docs
        ])
        if len(att_vals) < 5:
            print("🔁 Kişisel öneri için yeterli veri yok.")
            return

        prompt = f"""
🧠 Öğrenci dikkat geçmişi:
{attention_summary}

📊 Ortalama dikkat: {round(sum(att_vals)/len(att_vals), 2)}
🎙️ Ortalama focus: {round(sum(focus_vals)/len(focus_vals), 2)}

Bu öğrenciye uygun öğrenme periyodu ve stratejileri öner.
"""
        feedback = qa_chain.run(prompt)
        print("\n🧠 Öğrenme Önerisi:\n", feedback)
    except Exception as e:
        print(f"🚨 Öğrenme önerisi hatası: {e}")


def zihin_yorgunlugu_tahmini(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        if len(recent_docs) < 5:
            print("🔁 Yorgunluk analizi için yeterli veri yok.")
            return

        attention_values = [doc.metadata.get("attention", 0) for doc in recent_docs]
        first_half_avg = sum(attention_values[:3]) / 3
        last_half_avg = sum(attention_values[-3:]) / 3
        delta = first_half_avg - last_half_avg

        if delta > 0.15:
            print("🧠 Zihin yorgunluğu tespit edildi. Dikkatin ciddi oranda düştü.")
            try:
                import winsound
                winsound.Beep(600, 500)
            except:
                pass
            print("\033[93m[Yorgunluk Algılandı: Kısa bir ara vermelisin.]\033[0m")
    except Exception as e:
        print(f"🚨 Yorgunluk tahmin modülü hatası: {e}")


def vak_ogrenme_tarzi_tahmini(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        visual = 0
        auditory = 0
        kinesthetic = 0

        for doc in recent_docs:
            m = doc.metadata
            if m.get("screen"): visual += 1
            if m.get("focus_score", 0) > 0.6: auditory += 1
            if m.get("keyboard_activity") or m.get("mouse_activity"): kinesthetic += 1

        toplam = visual + auditory + kinesthetic
        if toplam == 0:
            print("🔁 Öğrenme tarzı için yeterli veri yok.")
            return

        style = max((visual, "Görsel"), (auditory, "İşitsel"), (kinesthetic, "Kinestetik"))[1]
        print(f"📚 Tahmini öğrenme tarzın: \033[94m{style}\033[0m")
    except Exception as e:
        print(f"🚨 Öğrenme tarzı modülü hatası: {e}")


def oturum_ozet_raporu(_):
    try:
        recent_docs = vectordb.similarity_search("attention", k=10)
        if not recent_docs:
            print("🔁 Özet için yeterli veri yok.")
            return

        avg_att = sum(doc.metadata.get("attention", 0) for doc in recent_docs) / len(recent_docs)
        avg_focus = sum(doc.metadata.get("focus_score", 0) for doc in recent_docs) / len(recent_docs)

        en_focused = max(recent_docs, key=lambda d: d.metadata.get("focus_score", 0))
        saat = en_focused.metadata.get("timestamp", "bilinmiyor")

        print("\n📊 \033[92mOturum Özeti:\033[0m")
        print(f"- Ortalama dikkat: {round(avg_att, 2)}")
        print(f"- Ortalama focus: {round(avg_focus, 2)}")
        print(f"- En odaklı zaman: {saat}")
    except Exception as e:
        print(f"🚨 Oturum özeti hatası: {e}")

def sesli_motivasyon_ver(mesaj="Harika gidiyorsun! Odaklanmaya devam et."):
    try:
        engine = pyttsx3.init()
        engine.setProperty('rate', 175)
        engine.say(mesaj)
        engine.runAndWait()
    except Exception as e:
        print(f"[Sesli Uyarı Hatası] {e}")

----------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------

In [ ]:
tools = [
    Tool(name="DikkatUyarisi", func=dikkat_uyarisi_ver, description="Dikkat seviyesi düşükse uyarı verir."),
    Tool(name="MolaOnerisi", func=mola_onerisi_chromadb, description="20dk ortalama dikkat düşüklüğünde mola önerir."),
    Tool(name="OgrenmePeriyoduOnerisi", func=periyot_onerisi_llm_ileri, description="Dikkat geçmişine göre öğrenme stratejisi önerir."),
    Tool(name="ZihinYorgunluguTahmini", func=zihin_yorgunlugu_tahmini, description="Son dikkat verilerindeki düşüşe göre zihin yorgunluğu tespit eder."),
    Tool(name="OgrenmeTarziTahmini", func=vak_ogrenme_tarzi_tahmini, description="Kullanıcının VAK öğrenme tarzını tahmin eder."),
    Tool(name="OturumOzeti", func=oturum_ozet_raporu, description="Son verilerle genel bir oturum özeti sunar.")
]


agent = initialize_agent(
    tools=tools,
    llm=LLM,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


----------------------------------------------------------------------------------------------------------------------

In [ ]:
def get_safe_json(url):
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200 and response.text.strip():
            return response.json()
        else:
            print(f"[UYARI] {url} boş ya da geçersiz yanıt döndü. Status: {response.status_code}")
            return {}
    except Exception as e:
        print(f"[HATA] {url} isteği başarısız: {e}")
        return {}


----------------------------------------------------------------------------------------------------------------------

In [ ]:
while True:
    # 📡 Veri kaynaklarından çek
    data = get_safe_json(ENDPOINT_ATTENTION)
    keyboard_data = get_safe_json(ENDPOINT_KYBRD_MOUSE_INTERRUPT)
    script_data = get_safe_json(ENDPOINT_SCRIPT)

    if not data:
        print("⏳ Veri alınamadı, bekleniyor...\n")
        time.sleep(5)
        continue

    # 🧾 Oturum bilgileri
    CURRENT_USER_ID = "ogrenci_001"
    CURRENT_SESSION_ID = datetime.now().strftime("%Y%m%d_%H")
    CURRENT_TIMESTAMP = datetime.now().isoformat()

    # 📊 Metadata
    metadata = {
        "user_id": CURRENT_USER_ID,
        "session_id": CURRENT_SESSION_ID,
        "timestamp": CURRENT_TIMESTAMP,
        "attention": data.get("attention", 0),
        "screen": data.get("head_looking_at_screen", False),
        "eye_left": data.get("left_eye_open", False),
        "eye_right": data.get("right_eye_open", False),
        "att_1min": data.get("attention_1min_avg", 0),
        "att_5min": data.get("attention_5min_avg", 0),
        "att_20min": data.get("attention_20min_avg", 0),
        "att_total": data.get("attention_total_avg", 0),
        "keyboard_activity": keyboard_data.get("keyboard_activity", False),
        "mouse_activity": keyboard_data.get("mouse_activity", False),
        "tab_changed": keyboard_data.get("tab_changed", False),
        "interaction_status": keyboard_data.get("status", "no_status"),
        "focus_score": script_data.get("focus_score", 0)
    }

    # 💾 Hafızaya ekle
    vectordb.add_texts(["Yeni öğrenci verisi alındı."], metadatas=[metadata])

    # 🧹 ChromaDB temizlik (max 300 kayıt)
    try:
        collection_data = vectordb._collection.get()
        doc_ids = collection_data.get("ids", [])
        if len(doc_ids) > 300:
            print("♻️ ChromaDB çok dolu, eski kayıtlar siliniyor...")
            vectordb._collection.delete(ids=doc_ids[:50])
    except Exception as e:
        print(f"[Chroma Temizleme Hatası] {e}")

    # 🎯 AGENT PROMPT'u: Öğrenciye destek olacak net ve koçvari
    prompt = f"""
Sen yapay zeka destekli profesyonel bir eğitim koçusun.
Aşağıda verilen öğrencinin dikkat, odak ve etkileşim verilerini analiz et.

Kişisel bir öneri ver: motivasyonel ya da yönlendirici olabilir.
Eğer dikkat çok düşükse mola öner, yorgunluk varsa belirt, focus varsa motive et.

Veriler:
- Dikkat: {metadata['attention']}
- 20dk Ort: {metadata['att_20min']}
- Focus: {metadata['focus_score']}
- Ekrana Bakıyor mu: {metadata['screen']}
- Klavye: {metadata['keyboard_activity']}, Mouse: {metadata['mouse_activity']}
- Sekme Değişimi: {metadata['tab_changed']}
- Durum: {metadata['interaction_status']}
"""

    print("\n🧠 Agent çalıştırılıyor...\n")
    try:
        agent.run(prompt)
    except Exception as e:
        print(f"[Agent Hatası] {e}")

    # ⏱️ 10 saniye sonra tekrar çalış
    time.sleep(10)


----------------------------------------------------------------------------------------------------------------------